In [4]:
from pandas import Timestamp
import numpy as np
from numba import njit
import numba
import pycurl_requests as requests
import pandas as pd
from pprint import pprint
import sys

sys.path.append(
    "/Users/GlebSokolov/.vscode/extensions/almenon.arepl-2.0.3/node_modules/arepl-backend/python"
)
# from arepl_dump import dump


In [5]:
import time
import numpy as np
import ccxt

symbol = "AGLD-USDT"

exchange = getattr(ccxt, "coinbase")()
since = "2021-10-05 10:00:00"
since_ts = exchange.parse8601(since)
end_ts = since_ts + (3600000)
end = str(pd.to_datetime(end_ts, unit="ms"))


In [6]:
@njit
def findid(lst, ts_start, ts_end, trade_ids):
    ids = []
    for i in range(68, -1, -1):
        for j in range(999):
            if ts_start <= lst[i, j] <= ts_end:
                ids.append((trade_ids[i, j], np.array([i, j])))
    return ids


In [7]:
headers = {
    "User-Agent": "python-requests/2.26.0",
    "Accept": "*/*",
    "Connection": "keep-alive",
}
responses = [
    requests.get(
        f"https://api.exchange.coinbase.com/products/AGLD-USDT/trades?limit=1000&after={i*1000}",
        headers=headers,
    ).json()
    for i in range(70)
]
responses.pop(0)


KeyboardInterrupt: 

In [ ]:
# %%snoop
trade_ids = np.zeros((69, 999), dtype=int)
rrr = np.zeros((69, 999))
for i in range(rrr.shape[0]):
    for j in range(rrr.shape[1]):
        rrr[i, j] = int(Timestamp(responses[i][j]["time"]).timestamp() * 1000)
        trade_ids[i, j] = responses[i][j]["trade_id"]



In [ ]:
trades = np.array(findid(rrr, since_ts, end_ts, trade_ids))
indexer = trades[:, 1]


/var/folders/wg/9dzl6b3523911b3hfyj83skr0000gn/T/ipykernel_9820/1531939820.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  trades = np.array(findid(rrr, since_ts, end_ts, trade_ids))


In [10]:
import coinbase
from coinbase.wallet.model import APIObject


In [11]:
from coinbase.wallet.client import Client

coinbase_API_key = "PZsUB9TdWedNjJcu"
coinbase_API_secret = "UP8Jz2cfXvDRdgeeNwFy6fCxfJCkQFea"
client = Client(coinbase_API_key, coinbase_API_secret)


In [12]:
%%capture
client.get_spot_price(currency='AGLD')
client._make_api_object(client._get('v2', 'prices', 'AGLD-USD', 'historic'), APIObject)

In [ ]:
# dir(public_client)

object_methods = [
    method_name
    for method_name in dir(public_client)
    if callable(getattr(public_client, method_name))
]
pprint(object_methods)


['__class__',
 '__delattr__',
 '__dir__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_send_message',
 '_send_paginated_message',
 'get_currencies',
 'get_product_24hr_stats',
 'get_product_historic_rates',
 'get_product_order_book',
 'get_product_ticker',
 'get_product_trades',
 'get_products',
 'get_time']


In [13]:
client._make_api_object(client._get('v2', 'prices', 'AGLD-USD', 'historic'), APIObject)

<APIObject @ 0x7fc34df437c0> {
  "base": "AGLD",
  "currency": "USD",
  "prices": [
    {
      "price": "1.6",
      "time": "2021-12-17T21:13:30Z"
    },
    {
      "price": "1.59",
      "time": "2021-12-17T21:13:20Z"
    },
    {
      "price": "1.6",
      "time": "2021-12-17T21:13:00Z"
    },
    {
      "price": "1.59",
      "time": "2021-12-17T21:12:10Z"
    },
    {
      "price": "1.61",
      "time": "2021-12-17T21:12:00Z"
    },
    {
      "price": "1.6",
      "time": "2021-12-17T21:11:10Z"
    },
    {
      "price": "1.59",
      "time": "2021-12-17T21:08:50Z"
    },
    {
      "price": "1.6",
      "time": "2021-12-17T21:08:20Z"
    },
    {
      "price": "1.6",
      "time": "2021-12-17T21:08:00Z"
    },
    {
      "price": "1.6",
      "time": "2021-12-17T21:05:20Z"
    },
    {
      "price": "1.6",
      "time": "2021-12-17T21:04:50Z"
    },
    {
      "price": "1.6",
      "time": "2021-12-17T21:04:10Z"
    },
    {
      "price": "1.59",
      "time": "2021

In [15]:
%%capture
import cbpro
public_client = cbpro.PublicClient()
public_client.get_product_ticker(product_id='AGLD-USD')

In [17]:
ttrades = public_client.get_product_trades(
    product_id=symbol, before=end_ts/1000, after=since_ts/1000, limit=100
)

In [18]:
ttrades[1]

TypeError: 'generator' object is not subscriptable

In [ ]:
data = public_client.get_product_historic_rates(
    start=since, end=end, product_id="AGLD-USDT", granularity=60
)
data = pd.DataFrame(data, columns=["time", "low", "high", "open", "close", "volume"])
data["time_iso"] = pd.to_datetime(data["time"], unit="s")



In [ ]:
data


,time,low,high,open,close,volume,time_iso
0,1633431300,3.64,3.69,3.64,3.69,127.96,2021-10-05 10:55:00
1,1633431180,3.59,3.59,3.59,3.59,231.58,2021-10-05 10:53:00
2,1633431000,3.63,3.66,3.66,3.63,44.16,2021-10-05 10:50:00
3,1633430760,3.65,3.65,3.65,3.65,240.38,2021-10-05 10:46:00
4,1633430400,3.66,3.66,3.66,3.66,11.49,2021-10-05 10:40:00
5,1633430340,3.67,3.68,3.67,3.68,70.02,2021-10-05 10:39:00
6,1633429980,3.65,3.74,3.74,3.65,526.11,2021-10-05 10:33:00
7,1633429740,3.74,3.74,3.74,3.74,254.25,2021-10-05 10:29:00
8,1633429680,3.75,3.75,3.75,3.75,558.48,2021-10-05 10:28:00
9,1633429500,3.73,3.74,3.73,3.74,834.72,2021-10-05 10:25:00


In [ ]:
responses[15][417]


TypeError: list indices must be integers or slices, not tuple

In [ ]:
a = [[15, 417]]
* a[0]



SyntaxError: can't use starred expression here (2949532561.py, line 2)

In [34]:
%%cython
import ciso8601, time
import cbpro
# public_client = cbpro.PublicClient()
# cdef int ts = 1633428000000
# cdef int te = 1633431600000
# resp = public_client.get_product_trades('AGLD-USDT')
def go_iter(gen, long int s, long int e):
    cdef long int[5000] idd
    cdef int num = 0
    cdef long int i = 0
    for i in gen:
        if s <= i and i <= e:
            idd[num] = num
    return idd

In [23]:
%load_ext Cython

In [35]:
def rtgen(resp):
    # cdef dict i
    for i in resp:
        rt = ciso8601.parse_datetime(i['time'])
        yield int(time.mktime(rt.timetuple())*1000)

In [37]:
rrt = rtgen(public_client.get_product_trades('AGLD-USDT'))
go_iter(rrt, since_ts, end_ts)

[0,
 140476734247152,
 140476788384032,
 140476828249664,
 140476835765024,
 140476678168928,
 140476835765024,
 140476835740544,
 140476807831664,
 32,
 140476807831664,
 0,
 -6526173486093107085,
 140476807831664,
 140476828249664,
 140476678168928,
 140476835765024,
 140476826910432,
 140701802272656,
 4547542288,
 4549956272,
 140476678168928,
 140476826910432,
 140476835765024,
 140476828249664,
 4547542016,
 140701802272752,
 4546927943,
 140476832698864,
 2,
 140476745684864,
 140476807831664,
 140476828249664,
 0,
 140476807831664,
 140476678168928,
 140476832698880,
 140701802273040,
 140701802272816,
 4548170423,
 24,
 140476822348992,
 141,
 140476746856994,
 140476745684864,
 3,
 140701802273232,
 4548161106,
 140476829025536,
 140476801211920,
 0,
 7,
 140476746957584,
 140476686567424,
 140476832698848,
 4620752421,
 140701802272992,
 140476805192384,
 140476746955040,
 140476832698432,
 140476686567492,
 140476832698536,
 140476678169016,
 140476746856928,
 1404766865674